<a href="https://colab.research.google.com/github/SanjaPanda/car-plate-detector/blob/main/Detecting_Lisence_Plate_Number.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing important libraries and packages

1. object-detection api
2. Tesseract - ocr
3. OCRopus

In [ ]:
%cd '/content/drive/MyDrive/car-plate-detector/models/research/'

!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

/content/drive/MyDrive/car-plate-detector/models/research
Processing /content/drive/My Drive/car-plate-detector/models/research
     |████████████████████████████████| 9.0MB 7.3MB/s 
     |████████████████████████████████| 358kB 54.2MB/s 
     |████████████████████████████████| 1.1MB 45.5MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 153kB 50.7MB/s 
     |████████████████████████████████| 2.2MB 54.1MB/s 
     |████████████████████████████████| 17.7MB 146kB/s 
     |████████████████████████████████| 829kB 56.7MB/s 
     |████████████████████████████████| 174kB 54.4MB/s 
     |████████████████████████████████| 102kB 12.9MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 645kB 50.4MB/s 
     |████████████████████████████████| 37.6MB 63kB/s 
     |████████████████████████████████| 706kB 49.3MB/s 
     |████████████████████████████████| 

In [ ]:
!sudo apt update
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev
!pip install pytesseract

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:15 http

In [ ]:
%cd '/content/drive/MyDrive/car-plate-detector/'

try:
  shutil.rmtree('ocropy')
  print('ocropy file deleted')
except:
  pass

!git clone https://github.com/ocropus/ocropy.git

/content/drive/MyDrive/car-plate-detector
Cloning into 'ocropy'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 4973 (delta 2), reused 0 (delta 0), pack-reused 4966
Receiving objects: 100% (4973/4973), 41.80 MiB | 18.73 MiB/s, done.
Resolving deltas: 100% (3294/3294), done.


In [ ]:
%cd '/content/drive/MyDrive/car-plate-detector/ocropy'
!sudo apt-get install $(cat PACKAGES)
!wget -nd https://github.com/zuphilip/ocropy-models/raw/master/en-default.pyrnn.gz
!mv en-default.pyrnn.gz models/
!sudo python2 setup.py install
!ocropus-nlbin --help

/content/drive/MyDrive/car-plate-detector/ocropy
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-lyx javascript-common libjs-jquery libjs-jquery-ui
  python-backports.functools-lru-cache python-bs4 python-chardet python-cycler
  python-dateutil python-decorator python-html5lib python-matplotlib-data
  python-olefile python-pil python-pkg-resources python-pyparsing python-six
  python-subprocess32 python-tz python-webencodings ttf-bitstream-vera
Suggested packages:
  apache2 | lighttpd | httpd libjs-jquery-ui-docs python-cycler-doc
  python-genshi python-lxml-dbg python-lxml-doc dvipng gir1.2-gtk-3.0
  ghostscript inkscape ipython python-cairocffi python-configobj
  python-excelerator python-gobject python-matplotlib-doc python-nose
  python-qt4 python-sip python-tornado python-traits python-wxgtk3.0
  texlive-extra-utils texlive-latex-extra ttf-staypuft python-pil-doc
  python-p

# Importing Model

In [ ]:
#Importing important libraries

import shutil
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import cv2
from PIL import Image as im
import matplotlib.pyplot as plt
import warnings
import pytesseract
import numpy as np
import glob

## Recognising Lisence Plate

In [ ]:
# Load saved model and build the detection function
detect_fn=tf.saved_model.load("/content/drive/MyDrive/car-plate-detector/workspace/training_demo/exported-models/my_model_ssd/saved_model")
print('Loaded Model')

category_index=label_map_util.create_category_index_from_labelmap("/content/drive/MyDrive/car-plate-detector/workspace/training_demo/annotations/label_map.pbtxt")

Loaded Model


In [ ]:
#Loading the image
%cd '/content/drive/MyDrive/car-plate-detector/'

warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

img_paths = [i for i in glob.glob('/content/drive/MyDrive/car-plate-detector/workspace/training_demo/images/test/*.png')]


/content/drive/MyDrive/car-plate-detector


In [ ]:
def load_image_into_numpy_array(path):
  
    img = cv2.imread(path)
    if img.shape[2] == 4:
      print('Changed Image')
      img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    return np.array(img), img

In [ ]:
def clean_img(imge):
  
  imge = cv2.cvtColor(imge, cv2.COLOR_BGR2GRAY)
  imge = cv2.resize(imge, None, fx = 3, fy = 3, interpolation = cv2.INTER_NEAREST)
  imge = cv2.GaussianBlur(imge, (5,5), 0) 
  imge = cv2.medianBlur(imge, 3)
  return imge

In [ ]:
def find_plate_num(imge):
  
  plate_num = pytesseract.image_to_string(imge, lang='eng', config='--psm 13 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ ')
  plate_num_clean = ''.join(map(str,[i.upper() for i in plate_num if i.isalnum() or i == ' ']))
  return plate_num_clean

In [ ]:
def plate_num_detection(image_path):
    pl_num = []
    coord = []
    thresh = []
    image_np, img = load_image_into_numpy_array(image_path)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor=tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor=input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections=detect_fn(input_tensor)

    # We're only interested in the first num_detections.

    num_detections=int(detections.pop('num_detections'))

    detections={key:value[0,:num_detections].numpy()
                   for key,value in detections.items()}

    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    count = 0
    plate_num_clean = ''
    while detections['detection_scores'][count] > 0.6:

      thresh.append(detections['detection_scores'][count])
      boxes = detections['detection_boxes'][count]
      height, width, channel = image_np.shape
      ymin = int((boxes[0]*height)+3)
      xmin = int((boxes[1]*width)+3)
      ymax = int((boxes[2]*height)-3)
      xmax = int((boxes[3]*width)-3)

      cropped_img = img[ymin:ymax, xmin:xmax]
      res_img = clean_img(cropped_img)
      cv2.imwrite('/content/drive/MyDrive/car-plate-detector/result/res_crop.png',res_img)
      
      !ocropus-nlbin -n result/res_crop.png -o result/bin_img > result/log1.txt
      res_img2 = cv2.imread('/content/drive/MyDrive/car-plate-detector/result/bin_img/0001.bin.png')

      plate_num = find_plate_num(res_img2)
      pl_num.append(plate_num)
      coord.append((xmin+5, ymin+5))
      
      count = count+1

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=count+5,     
          min_score_thresh=.4,   
          agnostic_mode=False)
    
    for n,i in enumerate(pl_num):
      image_np = cv2.putText(image_np,i,coord[n],cv2.FONT_HERSHEY_SIMPLEX ,1,(255, 255, 0),2, cv2.LINE_AA)

    return image_np, pl_num, thresh

In [ ]:
import pandas as pd

df = pd.DataFrame(columns = ['name','thresh','pred_num'])

In [ ]:
 for i in img_paths:
   img, plate_num,thresh = plate_num_detection(i)
   name = i[i.rfind('/')+1:]
   p = '/content/drive/MyDrive/car-plate-detector/result/test result/'+name
   cv2.imwrite(p,img)
   df = df.append({'name':name,'thresh':thresh,'pred_num':plate_num},ignore_index=True)
   #%matplotlib inline
   #plt.figure()
   #plt.imshow(img)
   #plt.show()

/usr/local/lib/python2.7/dist-packages/scipy/ndimage/interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
/usr/local/lib/python2.7/dist-packages/scipy/ndimage/interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
/usr/local/lib/python2.7/dist-packages/scipy/ndimage/interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
/usr/local/lib/python2.7/dist-packages/scipy/ndimage/interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of 

In [ ]:
df.to_csv('/content/drive/MyDrive/car-plate-detector/result/test result/result_val.csv')